In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle

import torch
import hydra
import omegaconf
import pyrootutils
from tqdm import tqdm
from glob import glob

import matplotlib.pyplot as plt
from fgvc.data.plant_clef_data import PlantCLEFDataset, PlantSPECIESDataset

In [10]:
df = pd.read_csv("/home/ubuntu/FGVC11/data/PlantClef/PlantCLEFTrainLQ.csv",  delimiter=";", escapechar="/")
df["vector_path"] = "/home/ubuntu/FGVC11/data/PlantClef/lq_feats/" + df["image_name"].apply(lambda x: x.replace(".jpg", ".pt"))
df.to_csv("/home/ubuntu/FGVC11/data/PlantClef/PlantCLEFTrainLQ.csv", sep=';', index=False, quoting=csv.QUOTE_NONE, escapechar='/')

/tmp/ipykernel_69598/583444022.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/ubuntu/FGVC11/data/PlantClef/PlantCLEFTrainLQ.csv",  delimiter=";", escapechar="/")


In [6]:
df.head()

,image_name,organ,species_id,obs_id,license,partner,author,altitude,latitude,longitude,...,genus,family,dataset,publisher,references,url,learn_tag,image_backup_url,path,vector_path
0,59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg,leaf,1396710,1008726402,cc-by-sa,NaN,Gulyás Bálint,205.9261,47.592160,19.362895,...,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/59feabe1c98f06...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,/home/ubuntu/FGVC11/data/PlantClef/images_trai...,/home/ubuntu/FGVC11/data/PlantClef/lq_feats/59...
1,dc273995a89827437d447f29a52ccac86f65476e.jpg,leaf,1396710,1008724195,cc-by-sa,NaN,vadim sigaud,323.7520,47.906703,7.201746,...,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/dc273995a89827...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,/home/ubuntu/FGVC11/data/PlantClef/images_trai...,/home/ubuntu/FGVC11/data/PlantClef/lq_feats/dc...
2,416235e7023a4bd1513edf036b6097efc693a304.jpg,leaf,1396710,1008721908,cc-by-sa,NaN,fil escande,101.3160,48.826774,2.352774,...,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/416235e7023a4b...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,/home/ubuntu/FGVC11/data/PlantClef/images_trai...,/home/ubuntu/FGVC11/data/PlantClef/lq_feats/41...
3,cbd18fade82c46a5c725f1f3d982174895158afc.jpg,leaf,1396710,1008699177,cc-by-sa,NaN,Desiree Verver,5.1070,52.190427,6.009677,...,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/cbd18fade82c46...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,/home/ubuntu/FGVC11/data/PlantClef/images_trai...,/home/ubuntu/FGVC11/data/PlantClef/lq_feats/cb...
4,f82c8c6d570287ebed8407cefcfcb2a51eaaf56e.jpg,leaf,1396710,1008683100,cc-by-sa,NaN,branebrane,165.3390,45.794739,15.965862,...,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/f82c8c6d570287...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,/home/ubuntu/FGVC11/data/PlantClef/images_trai...,/home/ubuntu/FGVC11/data/PlantClef/lq_feats/f8...


In [11]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/data/vector_plant_clef_data.yaml")
datamodule = hydra.utils.instantiate(cfg)
datamodule.setup()

/home/ubuntu/FGVC11/src/fgvc/data/plant_clef_data.py:387: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.df_path, delimiter=";", escapechar="/")


In [18]:
for batch in datamodule.train_dataloader():
    break

In [21]:
feats, labels = batch["vector"].unsqueeze(1), batch["encoded_label"]

In [22]:
feats.shape

torch.Size([16, 1, 768])

In [25]:
def reshape_tensor_labels(tensor, labels, N):
    # Original dimensions
    seq_len, feature_dim = tensor.size(1), tensor.size(2)
    _, num_classes = labels.size()

    # Reshape the tensor and labels
    reshaped_tensor = tensor.view(-1, N * seq_len, feature_dim)
    reshaped_labels = labels.view(-1, N, num_classes).sum(dim=1)
    reshaped_labels = (reshaped_labels > 0).float()
    # Convert to binary classification target

    return reshaped_tensor, reshaped_labels


In [26]:
feats, labels = reshape_tensor_labels(feats, labels, 4)

In [27]:
feats.shape, labels.shape

(torch.Size([4, 4, 768]), torch.Size([4, 7806]))

In [34]:
def find_divisors(N):
    """
    This function returns a list of numbers less than 20 that divide N without a remainder.

    :param N: The number for which to find divisors.
    :return: A list of divisors of N that are less than 20.
    """
    divisors = []
    for i in range(1, 25):  # Loop from 1 to 19
        if N % i == 0:
            divisors.append(i)
    return divisors

# Example usage
N = 600
find_divisors(N)

[1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 20, 24]